# Data Quality

This notebook shows some of the data quality functions associated with a {py:class}`Spectrum <dysh.spectra.spectrum.Spectrum>`. The data used for this example is taken from position switched observations of NGC5291 at 21 cm (AGBT05B_047_01).

The following {py:class}`Spectrum <dysh.spectra.spectrum.Spectrum>` functions will be discussed:

* {py:class}`stats <dysh.spectra.spectrum.Spectrum.stats>` : statistics of a spectrum
* {py:class}`roll <dysh.spectra.spectrum.Spectrum.roll>` : subtract the data by its rolled version to discover channel correlations and/or ripples in the spectrum
* {py:class}`radiometer <dysh.spectra.spectrum.Spectrum.radiometer>` : adherence of spectrum to the radiometer equation
* {py:class}`normalness <dysh.spectra.spectrum.Spectrum.normalness>` : p-value for null hypothesis that the data comes from a normal distribution
* {py:class}`snr <dysh.spectra.spectrum.Spectrum.snr>` : signal-to-noise ratio, either channel or flux based
* {py:class}`sratio <dysh.spectra.spectrum.Spectrum.sratio>` : flux ratio, a number between -1 and 1, if there is signal, 0 if none
* {py:class}`cog <dysh.spectra.spectrum.Spectrum.cog>` : curve of growth

<!--
```
stats         - statistics of a spectrum
roll          - a series of rolls (1,2,3...) to discover channel correlations and/or ripples in the spectrum
radiometer    - adherence of spectrum to the radiometer equation
snr           - signal/noise ratio, either channel or flux based
sratio        - flux ratio, an indiction of a number between -1 and 1, if there is signal, 0 if none.
normalness    - likelyhood noise is a gaussian, returned as a p-value  (p>0.05 means signal is gaussian(
cog           - curve of growth - can be useful to automatically determine where signal is
```
-->
## Loading Modules

We start by loading the modules we will use in this notebook. 

For display purposes, we use the static (non-interactive) `matplotlib` backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on `jupyter` lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl # Uncomment for interactive plots.

# These modules are required for the data reduction.
import numpy as np
from astropy import units as u
from dysh.util.files import dysh_data
from dysh.spectra.spectrum import Spectrum
from dysh.fits.gbtfitsload import GBTFITSLoad

from dysh.log import init_logging

In [ ]:
# Set logging to INFO level.
# This is only required in notebooks.
init_logging(2)

## Data Loading

We use the data from the tests for {py:class}`getps <dysh.fits.gbtfitsload.GBTFITSLoad.getps>` (AGBT05B_047_01).
We load the data using {py:class}`GBTFITSLoad <dysh.fits.gbtfitsload.GBTFITSLoad>`, and then its {py:class}`summary <dysh.fits.gbtfitsload.GBTFITSLoad.summary>` method to inspect its contents.

In [ ]:
filename = dysh_data(test="getps")      # AGBT05B_047_01/AGBT05B_047_01.raw.acs
sdfits = GBTFITSLoad(filename)

In [ ]:
sdfits.summary()

## Data Reduction

First we extract a (time-averaged) spectrum from the first scan. 
If you recall from the [Position-Switched example](https://dysh.readthedocs.io/en/latest/tutorials/examples/positionswitch.html), both edges have some problems, and for the purpose of this we remove some edge channels by slicing the {py:class}`Spectrum <dysh.spectra.spectrum.Spectrum>`. 
We end by plotting the resulting {py:class}`Spectrum <dysh.spectra.spectrum.Spectrum>` using channels for the x-axis.

In [ ]:
sp1 = sdfits.getps(scan=51, ifnum=0, plnum=0, fdnum=0).timeaverage()[5000:30000]
plot1 = sp1.plot(xaxis_unit='chan');

From the plot we see that there is a clear signal between channels 10000 and 13000.
We will need to avoid these channels in some of the examples below to reduce their impact on the computed data quality metrics.

## Statistics
We start by showing the use of the {py:class}`stats <dysh.spectra.spectrum.Spectrum.stats>` method.
This computes the mean, median, rms, mininum, maximum, number of points and number of points that are not-a-number (NaN).
We can compute these statistics over a subset of channels by slicing the {py:class}`Spectrum <dysh.spectra.spectrum.Spectrum>` before calling the method.
We do this for the first 10000 channels and the last 10000 channels of the previously obtained spectrum, not the full 32768 channel spectrum.

In [ ]:
# First 10,000 channels.
sp1[:10000].stats()

In [ ]:
# Last 10,000 channels.
sp1[-10000:].stats()

There is a significant continuum visible in the spectrum, with a value of ${\approx}270$ mK, but the two sides differ a bit, in the sense there is slight increase towards higher channels. This is also visible in the previous plot.

The rms levels are very similar, at 0.0712 K and 0.0719 K respectively.

It is possible to compute the statistics over a "rolled" version of the spectrum.
That is, before computing the statistics, the values from channel `i+roll` are subtracted from the values of channel `i`.
This accomplished using the `roll` argument in the call to {py:class}`stats <dysh.spectra.spectrum.Spectrum.stats>`.
When using ``roll!=0`` the rms is divided by $\sqrt{2}$ to account for the subtraction.

In [ ]:
print(sp1[:10000].stats(roll=2)["rms"])
print(sp1[-10000:].stats(roll=2)["rms"])

Here we see the rolled RMS is very similar to the direct RMS, a sign that there is little or no correlation between channels, and that there are no large scale variations in the baseline. This brings us to the next function, {py:class}`roll <dysh.spectra.spectrum.Spectrum.roll>`:

## Roll

The {py:class}`roll <dysh.spectra.spectrum.Spectrum.roll>` method compares the rms in the spectrum to the rms obtained after "rolling" the data by `roll` channels.
This method is useful for assessing if there are channel-to-channel correlations. In the abscence of correlations the result should be close to unity. 
Correlations will drive the results away from unity.
However, the results from {py:class}`roll <dysh.spectra.spectrum.Spectrum.roll>` are also affected by slow variations in the data, as is the case with this spectrum.
We thus compare the first and second half.

In [ ]:
sp1[:10000].roll(4)

In [ ]:
sp1[-10000:].roll(4)

Here one can argue there is no correllation between channels, and there is little large scale variation in the baseline. Note we have not subtracted a baseline, so we will need to do this again later in the notebook.

##  Radiometer Equation

For a given $T_{sys}$, channel width $\Delta f$ and observing time $\Delta t$ the radiometer equation predicts the expected noise as:
$$
  \Delta T = {  { T_{sys} } \over \sqrt{ \Delta f \Delta t } }
$$
The {py:class}`radiometer <dysh.spectra.spectrum.Spectrum.radiometer>` method will return the ratio of the measured noise to this expected noise.
In the abscence of artifacts, the result should be unity.
Deviations from unity might indicate that the noise properties do not follow those of a normal distribution.

For example, if we blindly compute this test we will get a value larger than one, because of line emission between channels 10000 and 13000.

In [ ]:
sp1.radiometer()

When we compute it on line-free channels the results are closer to unity.

In [ ]:
print(sp1[:10000].radiometer(), sp1[-10000:].radiometer())

However, the measured noise (rms) is still ${\approx}6\%$ larger than expected.

For comparison, we compute this statistic in the case of a purely Gaussian signal.
We do this using the {py:class}`fake_spectrum <dysh.spectra.spectrum.Spectrum.fake_spectrum>` method, which returns a spectrum filled with Gaussian noise.

In [ ]:
Spectrum.fake_spectrum(nchan=32768, seed=123).radiometer()

The exact value is a function of the number of channels in the spectrum, and it goes as $1/\sqrt{N}$. 
Experiments showed that for nchan=32768 the rms in this ratio is about 0.004.

## Normalness

The {py:class}`normalness <dysh.spectra.spectrum.Spectrum.normalness>` method gives the likelihood that the spectrum data is drawn from a normal distribution.
Under the hood this method computes the Anderson-Darling statistic and returns the p-value of the likelihood that the data is normally distributed.
A p-value larger than 0.05 indicates that the data is consistent with being drawn from a normal distribution.

We start by computing the normalness test for the line-free regions, and then we compute it for the whole spectrum.

In [ ]:
print(sp1[:10000].normalness())
print(sp1[-10000:].normalness())

These values are larger than 0.05, so the line-free regions are consistent with Gaussian noise.
For amusement, the whole spectrum:

In [ ]:
print(sp1.normalness())

In this case the p-value is lower than 0.05, indicating that the null-hyphotesis can be rejected, that is, the data is not drawn from a normal distribution.
That is the case because of the line between channels 10000 and 13000.

## Baseline Subtraction

The remaining data quality tests, the signal-to-noise ratio and signal ratio, only make sense if the data has no continuum.
In this case that requires subtracting a baseline from the data.

We know that there is a line between channels 10000 and 13000, so we exclude these channels from the baseline fit using the `exclude` argument of {py:class}`Spectrum.baseline <dysh.spectra.spectrum.Spectrum.baseline>`.
We saw earlier that there's a slope to the data, so we use a degree 2 polynomial for the baseline model, to to ensure any curvature is removed as well. Again, these things can later be tested.
We remove the best fit polynomial model from the data (`remove=True`).

In [ ]:
sp1.baseline(degree=2, model="poly", exclude=[10000,13000], remove=True)

Plot the baseline subtracted spectrum and print its statistics.

In [ ]:
sp1.plot(xaxis_unit='chan');

In [ ]:
sp1[:10000].stats()

In [ ]:
sp1[-10000:].stats()

## snr: Signal-to-Noise Ratio

The {py:class}`snr <dysh.spectra.spectrum.Spectrum.snr>` method computes the signal-to-noise ratio of the spectrum.
This can be done in two ways: channel based and flux based.
For the latter a section of the spectrum where the signal is expected has to be selected.
For the channel based analysis, it will compute the ratio between the maximum value (``peak=True``, default) and the noise.
For an absorption signal one can use ``peak=False`` where the minimum is used instead.

For pure noise in a channel based comparision (the default) we would expect the signal-to-noise to be around 4, higher for higher number of channels, as can be computed via the error function.

In [ ]:
print(sp1[:10000].snr(), sp1[-10000:].snr())

Now we repeat the ``snr`` call, but using the flux instead.
We show the results for the line-free regions of the spectrum and for the section with the line.

In [ ]:
print(sp1[:10000].snr(flux=True))
print(sp1[-10000:].snr(flux=True))
print(sp1[10000:13000].snr(flux=True))

The line-free regions have a signal-to-noise of $<0.5$, so no signal in them.
Channels 10000 to 13000 have a signal-to-noise ratio of $47$. Really it should be called flux-to-error ratio.

Applying this to the whole spectrum, we still get a respectable ratio of over 5:

In [ ]:
sp1.snr()

## sratio:  Signal Ratio

The {py:class}`snr <dysh.spectra.spectrum.Spectrum.sratio>` signal ratio is defined as the sum between the positive and negative sum of the signals, normalized by the difference of both. This results in a dimensionless number between -1 and 1, where -1 means a pure absorption signal, 0 pure noise, and 1 pure emission line:
$$
     S_r = {{ P_{sum}+ N_{sum} } \over {P_{sum} - N_{sum}}}
$$
with $P_{sum}$ the sum of all the channels with positive values and $N_{sum}$ the sum of all the channels with negative values.
Note that $N_{sum}$ is negative.

We compute this for the line-free channels, where we expect values close to zero (no signal).

In [ ]:
print(sp1[:10000].sratio())

In [ ]:
print(sp1[-10000:].sratio())

And now the channels with the line.

In [ ]:
print(sp1[10000:13000].sratio())

In this case we have a value of $0.74$, close to unity, indicating an emission line.

## Repeat statistics, roll and radiometer

Now that the spectrum has been baseline subtracted we can repeat the analysis carried out above.

In [ ]:
# redo the tests
print(sp1[:10000].stats())
print(sp1[:10000].stats(roll=1))
print("ROLL",sp1[:10000].roll(4))


print(sp1[-10000:].stats())
print(sp1[-10000:].stats(roll=1))
print("ROLL",sp1[-10000:].roll(4))

In [ ]:
print(sp1[:10000].radiometer(), sp1[-10000:].radiometer())

Curious how the radiometer equation holds as function of time?

In [ ]:
for scan in [51,53,55,57]:
    sp2 = sdfits.getps(scan=scan, ifnum=0, plnum=0, fdnum=0).timeaverage()[5000:30000]
    print(scan, sp2[:10000].radiometer(), sp2[-10000:].radiometer())

So we can conclude it did not change behavior during this observation, but there is a clear 5-6% deviation from an ideal telescope.

## Smoothing

Smoothing should give us a much clearer detection.
We smooth the data using a Hanning kernel with a width of 50 channels.
Lets see how the previous measures live of to this task.

In [ ]:
n = 50
method = 'hanning'
sp1s = sp1.smooth(method, n)

sp1s.plot(xaxis_unit='chan')

The line is now between channels 200 and 260.

Lets print the rms for this smoothed signal. 
Unsmoothed we found 0.07 K so we should expect $\sqrt{50}$ better, or about 0.01 K.

In [ ]:
print(sp1s[:200].stats()["rms"], sp1s[-200:].stats()["rms"])

Smoothing reduced the noise as expected.

Now the radiometer equation test.

In [ ]:
print(sp1s[:200].radiometer())
print(sp1s[-200:].radiometer())

In this case the test suggests that the data is worse than before (it was $5\%$ higher, now it is $11\%$).
This could be an artifact of ``dysh``, and is being investigated in issue 917

Now the signal-to-noise ratio.

In [ ]:
print("Signal-to-Noise by channel")
print(sp1s[:200].snr(peak=True), sp1s[:200].snr(peak=False), 'left side')
print(sp1s[-200:].snr(peak=True), sp1s[-200:].snr(peak=False), 'right side')
print(sp1s[200:260].snr(peak=True), sp1s[200:260].snr(peak=False), 'central signal portion')

In [ ]:
print("Signal-to-Noise by flux")
print(sp1s[:200].snr(flux=True), 'left side')
print(sp1s[-200:].snr(flux=True), 'right side')
print(sp1s[200:260].snr(flux=True), 'central signal portion')

Notice that the signal-to-noise in the line intensity is now lower than before (it was ${\approx}47$).
That is because the ``snr`` method uses the data itself to determine the noise.
Since we are using a channel range that mostly contains signal, the noise estimate is higher, thus reducing the signal-to-noise ratio.
We can give our own estimate of the noise using the ``rms`` parameter when calling ``snr``.
Like

In [ ]:
print(sp1s[200:260].snr(flux=True, rms=sp1s[-200:].stats()["rms"]), 'center signal portion with given rms')

This is closer to the previous estimate.
It also shows that smoothing the data does not improve the signal-to-noise ratio on the line intensity for this velocity resolved line.

And the signal ratio.

In [ ]:
sp1s[200:260].sratio()

a value very close to 1, so a clear signal.

This brings up the question of statistical significance. Unlike the `normalness()` function, we don't get a p-value out of this. We would have to compute this ourselves.

Without much more discussion, we offer the following experiment. Be aware, this cell can take a few minutes to compute depending on the value of `m`:

In [ ]:
%%time 
m=1000                # number of experiments
d=np.zeros(m)         # array containing the sratio values
n=32768               # largest size of spectrum


for nchan in [n, n//4, n//4//4, n//4//4//4]:
    for i in range(m):
        sp = Spectrum.fake_spectrum(nchan=nchan, seed=i, use_wcs=False)
        sp.data = sp.data - 0.1     # current fake_spectrum has hardcoded mean 0.1
        d[i] = sp.sratio()
    print(d.mean(),d.std(),nchan,m)

In [ ]:
# plot histogram of last nchan loop with 512 channels
import matplotlib.pyplot as plt
plt.hist(d, bins="auto");
plt.xlabel("sratio value");
plt.ylabel("Counts");

The RMS follows the expected $1/\sqrt{N}$ behavior, and for given $N$ one can assign a p-value based on the error function if this distribution is normal.  We leave this as an excersize for the reader.

## Curve of Growth

The [Curve of Growth method](https://dysh.readthedocs.io/en/latest/explanations/cog/index.html) ([Yu et al. 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...898..102Y/abstract)) also lists lots of interesting properties that can be used in some of the quality assessment functions we have discussed above.

We merely show the result of the `cog` function here, and plot the spectrum now in units of km/s:

In [ ]:
sp1s.cog()

In [ ]:
sp1s.plot(xaxis_unit='km/s')